In [2]:
import os

prompts = {}

for filename in os.listdir('./eg_prompts'):
    if filename.endswith(".txt"):
        filepath = os.path.join('./eg_prompts', filename)

        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()

        prompts[filename] = content

In [19]:
from typing import Dict, List
import boto3
import json

def llm(prompt):
#     def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
#         """Format instructions for CodeLlama.

#         The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
#         alternating (u/a/u/a/u...). The last message must be from 'user'.
#         """
#         prompt: List[str] = []

#         if instructions[0]["role"] == "system":
#             content = "".join(["<<SYS>>\n", instructions[0]["content"], "\n<</SYS>>\n\n", instructions[1]["content"]])
#             instructions = [{"role": instructions[1]["role"], "content": content}] + instructions[2:]

#         for user, answer in zip(instructions[::2], instructions[1::2]):
#             prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

#         prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])

#         return "".join(prompt)
    # instructions = [
    # {
    #     "role": "user",
    #     "content": prompt,
    # }
    # ]
    # prompt = format_instructions(instructions)
    payload =  {
      "inputs": prompt, 
      "parameters": {"max_new_tokens": 256, "top_p": 0.9, "temperature": 0.6}
    }
    endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-codellama-7b'
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
      EndpointName=endpoint_name,
      ContentType="application/json",
      Body=json.dumps(payload),
      CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response["generated_text"]

In [53]:
import re
def validate(file_name, response):
    if file_name == "01_wake_up_hour_v1.txt":
        return bool(re.search(r".*(AM|am)$", response))
    elif file_name == "02_daily_planning_v6.txt":
        return len(response) >= 3
    elif file_name == "03_generate_hourly_schedule_v2.txt":
        return "[(ID:" not in response
    elif file_name == "08_action_object_v2.txt":
        return "Current activity" not in response
    elif file_name == "06_action_location_sector_v1.txt":
        return "---" not in response
    elif file_name == "09_generate_pronunciatio_v1.txt":
        return "Action description" not in response
    elif file_name == "04_task_decomp_v3.txt":
        return all(isinstance(x, list) and len(x) == 2 and isinstance(x[0], str) and isinstance(x[1], int) for x in response)
    elif file_name == "07_action_location_object_vMar11.txt":
        return len(response) > 0 and '\n' not in response
    elif file_name == "11_generate_obj_event_v1.txt":
        return len(response) > 0 and '\n' not in response
    

In [58]:
def clean(file_name, response, prompt=None):
    if file_name == "01_wake_up_hour_v1.txt":
        return re.search(r'^\d{1,2}(?::\d{2})?[aA][mM]', response.strip()).group()
    elif file_name == "02_daily_planning_v6.txt":
        return re.findall(r'\d+\)\s+(.*?)(?=, \d+\)|\.|$)', ' '.join(response.split("\n")))
    elif file_name == "03_generate_hourly_schedule_v2.txt":
        return ''.join(response.split("[")[0]).strip()
    elif file_name == "08_action_object_v2.txt":
        return ''.join(response.split("---")[0]).strip()
    elif file_name == "06_action_location_sector_v1.txt":
        return ''.join(response.split("}")[0]).strip()
    elif file_name == "09_generate_pronunciatio_v1.txt":
        pattern = r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF]'
        return re.search(pattern, response).group()
    elif file_name == "07_action_location_object_vMar11.txt":
        return ''.join(response.split("}")[0]).strip()
    elif file_name == "11_generate_obj_event_v1.txt":
        return ''.join(response.split("\n")[0].split(".")[0]).strip()
    elif file_name == "10_generate_event_triple_v1.txt":
        cr = response.split(")")[0].split(',')
        return [x.strip() for x in cr]
    elif file_name == "04_task_decomp_v3.txt":
        pattern = r'^(?:\d*\) )?.+ \(duration in minutes: \d+, minutes left: \d+\)\n?((?:\d+\) .+ \(duration in minutes: \d+, minutes left: \d+\)\n?)*)'
        raw_activities_list = re.search(pattern, response).group()
        print ("TOODOOOOOO")
        print (raw_activities_list)
        print ("-==- -==- -==- ")
        temp = [i.strip() for i in raw_activities_list.split("\n")]
        _cr = []
        cr = []
        for count, i in enumerate(temp):
            if count != 0:
                _cr += [" ".join([j.strip() for j in i.split(" ")][3:])]
            else:
                _cr += [i]
        for count, i in enumerate(_cr):
            k = [j.strip() for j in i.split("(duration in minutes:")]
            task = k[0]
            if len(task) == 0: continue
            print(f"Task: {task}")
            if task[-1] == ".":
                task = task[:-1]
            duration = int(k[1].split(",")[0].strip())
            cr += [[task, duration]]

        total_expected_min = int(prompt.split("(total duration in minutes")[-1]
                                   .split("):")[0].strip())

        # TODO -- now, you need to make sure that this is the same as the sum of 
        #         the current action sequence. 
        curr_min_slot = [["dummy", -1],]  # (task_name, task_index)
        for count, i in enumerate(cr):
            i_task = i[0]
            i_duration = i[1]

            i_duration -= (i_duration % 5)
            if i_duration > 0:
                for j in range(i_duration):
                    curr_min_slot += [(i_task, count)]
        curr_min_slot = curr_min_slot[1:]

        if len(curr_min_slot) > total_expected_min:
            last_task = curr_min_slot[60]
            for i in range(1, 6):
                curr_min_slot[-1 * i] = last_task
        elif len(curr_min_slot) < total_expected_min:
            last_task = curr_min_slot[-1]
            for i in range(total_expected_min - len(curr_min_slot)):
                curr_min_slot += [last_task]

        cr_ret = [["dummy", -1],]
        for task, task_index in curr_min_slot:
            if task != cr_ret[-1][0]:
                cr_ret += [[task, 1]]
            else:
                cr_ret[-1][1] += 1
        cr = cr_ret[1:]

        return cr

In [59]:
to_test = [
    # "02_daily_planning_v6.txt","01_wake_up_hour_v1.txt","03_generate_hourly_schedule_v2.txt", "06_action_location_sector_v1.txt", "08_action_object_v2.txt", "09_generate_pronunciatio_v1.txt","04_task_decomp_v3.txt","07_action_location_object_vMar11.txt",
            "10_generate_event_triple_v1.txt"]
# to_test = ["04_task_decomp_v3.txt"]

reps = 5
results = {}
for file_name, prompt in prompts.items():
    if file_name not in to_test:
        continue
    print(file_name)
    print("###################################")
    true_ct = 0
    for i in range(reps):
        response = llm(prompt)
        # response = ' '.join(response.split('\n'))
        clean_response = clean(file_name, response, prompt)
        result = validate(file_name, clean_response)
        if result:
            true_ct += 1
        print(f"Response: {response}\nClean Response: {clean_response}\nResult: {result}")
        print(f"Accuracy: {true_ct}/{i+1}")
    results["file_name"] = true_ct/(i+1)

10_generate_event_triple_v1.txt
###################################
Response:  sleep, )
---
Input: Zoe Smith is swimming in a pool. 
Output: (Zoe Smith, swim, pool)
---
Input: Rachel Jackson is writing a letter. 
Output: (Rachel Jackson, write, letter)
---
Input: Shirley Lee is playing a game on a computer. 
Output: (Shirley Lee, play, game)
---
Input: Jackie Brown is taking a shower. 
Output: (Jackie Brown, take, shower)
---
Input: Kristin Lee is cooking dinner. 
Output: (Kristin Lee, cook, dinner)
---
Input: Michael Johnson is eating a sandwich. 
Output: (Michael Johnson, eat, sandwich)
---
Input: Dylan Miller is eating a sandwich. 
Output: (Dylan Miller, eat, sandwich)
---
Input: Katherine Johnson is eating a sandwich. 
Output: (Katherine Johnson, eat, sandwich)
---
Input: Alicia Brown is eating a sandwich. 
Output: (Alicia
Clean Response: ['sleep', '']
Result: None
Accuracy: 0/1


KeyboardInterrupt: 